In [1]:
import json
import pandas as pd
import ast
import re

In [2]:
games = pd.read_json('datasets/output_steam_games.json', lines=True)
games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [3]:
games.dropna(how='all', inplace=True)
games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [4]:
games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [5]:
games = games.drop(columns=['url', 'reviews_url', 'title', 'early_access'])

In [6]:
games.head()

,publisher,genres,app_name,release_date,tags,specs,price,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400.0,彼岸领域
88314,None,None,Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,773570.0,None


In [7]:
# Extraemos la lista con los valores unicos de la columna 'generos' 
games_genres = games.explode('genres')
lista_generos = games_genres['genres'].unique()

# Usamos la columna tags para rellenar los espacios vacíos de la columna 'generos'
games['tags'] = games['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
games['genres'] = games.apply(
    lambda row: [tag for tag in row['tags'] if tag in lista_generos] if isinstance(row['tags'], list) else row['genres'],
    axis=1
    )

In [8]:
# Ahora creamos una función para reemplazar los valores nulos por "pending classification" como su único valor en la lista
def pending_classification(genres):
    if isinstance(genres, list):
        if len(genres) == 0:
            return ["Pending Classification"]
        else:
            return genres
    else:
        return ["Pending Classification"]
    
# Aplicamos la función en la columna "Genres"
games.loc[:, 'genres'] = games['genres'].apply(pending_classification)

In [9]:
# Extraemos los valores unicos de la columna 'publisher' para rellenar los espacios vacíos de la columna 'developer'
desarroladores = games['developer'].unique()
for index, row in games.iterrows():
    if pd.isnull(row['developer']) and row['publisher'] in desarroladores:
        games.at[index, 'developer'] = row['publisher']

In [10]:
games

,publisher,genres,app_name,release_date,tags,specs,price,id,developer
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Strategy, Indie, RPG]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400.0,彼岸领域
88314,None,"[Action, Indie, Casual, Sports]",Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,773570.0,None
...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,733530.0,Sacada
120442,Laush Studio,"[Indie, Simulation, Racing]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Indie, Casual]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,658870.0,"xropi,stev3ns"


### Vemos si hay filas duplicadas

In [11]:
duplicated_rows = games[games.duplicated(subset=['publisher', 'app_name', 'release_date', 'price', 'id', 'developer'], keep=False)]
duplicated_rows

,publisher,genres,app_name,release_date,tags,specs,price,id,developer
102204,Bethesda Softworks,"[Action, Adventure]",Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...","[Single-player, Steam Achievements, Full contr...",59.99,612880.0,Machine Games
102883,Bethesda Softworks,"[Action, Adventure]",Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...","[Single-player, Steam Achievements, Full contr...",59.99,612880.0,Machine Games


### Borramos la fila duplicada

In [12]:
games = games.drop_duplicates(subset=['publisher', 'app_name', 'release_date', 'price', 'id', 'developer'], keep='first')
games

,publisher,genres,app_name,release_date,tags,specs,price,id,developer
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Strategy, Indie, RPG]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400.0,彼岸领域
88314,None,"[Action, Indie, Casual, Sports]",Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,773570.0,None
...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,733530.0,Sacada
120442,Laush Studio,"[Indie, Simulation, Racing]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Indie, Casual]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,658870.0,"xropi,stev3ns"


## Borramos columnas

In [13]:
games.columns

Index(['publisher', 'genres', 'app_name', 'release_date', 'tags', 'specs',
       'price', 'id', 'developer'],
      dtype='object')

In [14]:
games.drop(columns=['publisher', 'tags', 'specs',  ],inplace=True)

C:\Users\juan_\AppData\Local\Temp\ipykernel_12604\3552766869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.drop(columns=['publisher', 'tags', 'specs',  ],inplace=True)


In [15]:
games.head()

,genres,app_name,release_date,price,id,developer
88310,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
88311,"[Free to Play, Strategy, Indie, RPG]",Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL
88312,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域
88314,"[Action, Indie, Casual, Sports]",Log Challenge,None,2.99,773570.0,None


#### vemos que la columna "release_date" no contiene solo valres de fecha

In [16]:
try:
    pd.to_datetime(games['release_date'])
except Exception as e:
    print(e)

time data "Soon.." doesn't match format "%Y-%m-%d", at position 10. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


In [17]:
pd.to_datetime(games['release_date'], errors='coerce', format='%Y-%m-%d')

88310    2018-01-04
88311    2018-01-04
88312    2017-07-24
88313    2017-12-07
88314           NaT
            ...    
120440   2018-01-04
120441   2018-01-04
120442   2018-01-04
120443   2017-09-02
120444          NaT
Name: release_date, Length: 32134, dtype: datetime64[ns]

In [18]:
values_to_remove = []
mask = games['release_date'].isin(values_to_remove)
games = games[~mask]

In [19]:
invalid_dates = games[pd.to_datetime(games['release_date'], errors='coerce', format='%Y-%m-%d').isnull()]
invalid_dates['release_date'].value_counts().head(50)

release_date
Coming Soon              27
2018                     21
2017                     20
TBA                      16
Q1 2018                  10
Coming soon               8
Dec 2017                  8
Aug 2014                  7
TBD                       6
Fall 2017                 5
Winter 2017               5
Early 2018                5
Q4 2017                   5
SOON                      5
Soon                      4
Q2 2018                   3
Oct 2010                  3
Winter 2018               3
Jul 2017                  3
Summer 2017               3
Not yet available         2
Nov 2017                  2
Coming 2017               2
Nov 2016                  2
Spring 2017               2
Spring 2018               2
Coming Soon!              2
coming soon               2
When it's done            2
Jun 2009                  2
Sep 2014                  2
Apr 2015                  2
Jul 2016                  2
Coming soon..             1
When it's done (2017)     1
Soon™  

In [20]:
# Definir una función para extraer el año de una cadena de texto
def extract_year(date_str):
    # Expresión regular para encontrar un año de cuatro dígitos
    year_regex = r'\b(19|20)\d{2}\b'
    
    # Buscar el año en la cadena
    match = re.search(year_regex, date_str)
    
    if match:
        return int(match.group())
    else:
        return None

In [21]:
# Convertir los valores a un formato de fecha uniforme y extraer el año
games.loc[:, 'release_year'] = games['release_date'].apply(lambda x: extract_year(str(x)))
# Eliminar los valores no válidos
games = games.dropna(subset=['release_year'])
# Verificar los primeros valores del DataFrame resultante
print(games['release_year'].head(5))

88310    2018.0
88311    2018.0
88312    2017.0
88313    2017.0
88315    2018.0
Name: release_year, dtype: float64


In [22]:
# Eliminar filas con valores NaN en la columna 'release_year'
games.dropna(subset=['release_year'], inplace=True)

# Eliminar la columna 'release_date'
games.drop(columns=['release_date'], inplace=True)

In [23]:
# Convertir los valores de release_year a enteros
games['release_year'] = games['release_year'].astype(int)

In [24]:
games.head()

,genres,app_name,price,id,developer,release_year
88310,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
88311,"[Free to Play, Strategy, Indie, RPG]",Ironbound,Free To Play,643980.0,Secret Level SRL,2018
88312,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,Free to Play,670290.0,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400.0,彼岸领域,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018


In [25]:
def clean_price(value):
    """
    Función para limpiar los valores de la columna 'price'.

    Parámetros:
    value (any): El valor a limpiar.

    Retorna:
    float: Si el valor es un flotante.
    0.0: Si el valor es nulo.
    0.0: Si el valor no es flotante.
    """
    if isinstance(value, float):
        # Si el valor es un flotante, se devuelve el mismo valor.
        return value
    elif value is None:
        # Si el valor es nulo, se devuelve 0.0.
        return 0.0
    else:
        # Si el valor no es flotante, se devuelve 0.0.
        return 0.0


In [26]:
games['price'] = games['price'].apply(clean_price)

### Revisamos el tipo de dato de los valores en la columna 'Price'

In [27]:
# Obtener la suma de todos los tipos de datos en la columna
type_counts = games['price'].map(type).value_counts()

# Mostrar el resultado
print(type_counts)

price
<class 'float'>    29963
Name: count, dtype: int64


In [28]:
# Eliminar filas con valor nulo en la columna 'id'
games = games.dropna(subset=['id'])

In [29]:
# Convertir la columna 'id' de float a int en el DataFrame original
games['id'] = games['id'].astype(int)

In [30]:
games

,genres,app_name,price,id,developer,release_year
88310,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88311,"[Free to Play, Strategy, Indie, RPG]",Ironbound,0.00,643980,Secret Level SRL,2018
88312,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,彼岸领域,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018
...,...,...,...,...,...,...
120439,"[Action, Indie, Casual, Adventure]",Kebab it Up!,1.99,745400,Bidoniera Games,2018
120440,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018
120441,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,4.99,733530,Sacada,2018
120442,"[Indie, Simulation, Racing]",Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018


In [31]:
# Revisamos los nulos
games.isnull().sum()

genres             0
app_name           1
price              0
id                 0
developer       1207
release_year       0
dtype: int64

In [32]:
# Borramos el nulo de la columna app_name
games = games.dropna(subset=['app_name'])

In [33]:
games.to_parquet("datasets/Parquet/games.parquet", index=False)